In [5]:
import pandas as pd
import numpy as np
from threading import Thread
from tqdm import tqdm
import cv2
import tensorflow as tf
import tensorflow.keras.layers as layers
from sklearn.preprocessing import LabelEncoder


In [3]:
'''
import shutil
'''
# To copy files
shutil.copyfile('/content/drive/MyDrive/DFDC_AnirudhJoshi', 'destination_file_location')

# To copy multiple files of folder with some conditions
import os
for file in os.listdir('folder_path'):
    if file.endswith('.jpg'):
        shutil.copyfile('folder_path' + file, 'desination_folder_path' + file)
'''
# To copy folders
shutil.copytree('/content/drive/MyDrive/DFDC_AnirudhJoshi', '/content/sample_data/dfdc')  
'''

'/content/sample_data/dfdc'

In [4]:
import pickle
with open('/content/sample_data/dfdc/dfdc.pkl','rb') as f:
  data = pickle.load(f)

In [5]:
from sklearn.model_selection import train_test_split

Meso-Net

In [6]:
def convolution(model,n_f,f,p):
    ''' Function Parameters
        model : Pass the Neural Network
        n_f : number of filters
        f : filter matrix len
        p : max pool matrix len
    '''
    model.add(layers.Conv2D(n_f,(f,f),padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D((p,p),strides=2))

In [7]:
def fully_connected(model,drop_prob,nodes):
    ''' Function Parameters
        model : Pass the Neural Network
        drop_prob : for dropout
        nodes : # nodes in layer
    '''
    model.add(layers.Dropout(drop_prob))
    model.add(layers.Dense(nodes))

In [8]:
def meso(X_train,y_train,X_test,y_test):
    # X_train , y_train , X_test , y_test
    model = tf.keras.Sequential()
    model.add(layers.InputLayer((160, 160, 3,)))
    convolution(model,8,3,2)
    convolution(model,8,5,2)
    convolution(model,16,5,2)
    convolution(model,16,5,4)
    model.add(layers.Flatten())
    fully_connected(model,0.5,16)
    fully_connected(model,0.5,1)
    model.add(layers.Activation('sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    model.summary()
    model.fit(X_train,y_train,epochs=20,batch_size=256,verbose=1)
    loss,acc = model.evaluate(X_test,y_test)
    print("Loss : ",loss," Accuracy : ",acc)

Meso-Inception Net

In [9]:
def incept(layer,x,y,z,w):
    a = layers.Conv2D(x,(1,1),padding='same')(layer)
    b1 = layers.Conv2D(y,(1,1),padding='same',activation='relu')(layer)
    b2 = layers.Conv2D(y,(3,3),padding='same')(b1)
    b = layers.concatenate([b1,b2])
    c1 = layers.Conv2D(z,(1,1),padding='same',activation='relu')(layer)
    c2 = layers.Conv2D(z,(3,3),padding='same',dilation_rate=(2,2))(c1)
    c = layers.concatenate([c1,c2])
    d1 = layers.Conv2D(w,(1,1),padding='same',activation='relu')(layer)
    d2 = layers.Conv2D(w,(3,3),padding='same',dilation_rate=(3,3) )(d1)
    d = layers.concatenate([d1,d2])
    layer_out = layers.concatenate([a,b,c,d])
    return layer_out

In [10]:
def meso_inception(X_train,X_test,y_train,y_test):
    layer = layers.Input(shape=(160,160,3))
    layer_out = incept(layer,1,4,4,1)
    layer_out = incept(layer_out,1,4,4,2)
    layer_out = layers.Flatten()(layer_out)
    layer_out = layers.Dense(1,activation='relu')(layer_out)
    model = tf.keras.Model(inputs=layer,outputs=layer_out)
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    model.summary()
    model.fit(X_train,y_train,epochs=10,batch_size=256,verbose=1)
    loss,acc = model.evaluate(X_test,y_test)
    print("Loss : ",loss," Accuracy : ",acc)

RESNET-50 with spatial pyramidal pooling [DSP-FWA]

In [11]:
pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 8.6 MB/s eta 0:00:01


In [12]:
from tensorflow_addons import layers as lay

In [13]:
def identity_block(base,f1,f2):
    
    x = base
    
    base = layers.Conv2D(f1,(1,1),padding='valid')(base)
    base = layers.BatchNormalization()(base)
    base = layers.Activation('relu')(base)
    
    base = layers.Conv2D(f1,(3,3),padding='same')(base)
    base = layers.BatchNormalization()(base)
    base = layers.Activation('relu')(base)
    
    base = layers.Conv2D(f2,(1,1),padding='valid')(base)
    base = layers.BatchNormalization()(base)
    
    base = layers.Add()([base,x])
    base = layers.Activation('relu')(base)

    return base

In [14]:
def convolutional_block(base,s,f1,f2):
    
    x = base
    
    base = layers.Conv2D(f1,(1,1),strides=(s,s),padding='valid')(base)
    base = layers.BatchNormalization()(base)
    base = layers.Activation('relu')(base)
    
    base = layers.Conv2D(f1,(3,3),padding='same')(base)
    base = layers.BatchNormalization()(base)
    base = layers.Activation('relu')(base)
    
    base = layers.Conv2D(f2,(1,1),padding='valid')(base)
    base = layers.BatchNormalization()(base)
    
    x = layers.Conv2D(f2,(1,1),strides=(s,s),padding='valid')(x)
    x = layers.BatchNormalization()(x)
    
    base = layers.Add()([base,x])
    base = layers.Activation('relu')(base)
        
    return base

In [15]:
def ResNet50(X_train,y_train,X_test,y_test):
    input_layer = layers.Input(shape=(160,160,3))
    
    x = layers.ZeroPadding2D(padding=(3,3))(input_layer)
    x = layers.Conv2D(64,(7,7),strides=(2,2))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((3,3),strides=(2,2))(x)
    
    x = convolutional_block(x,1,32,64)
    x = identity_block(x,32,64)
    x = identity_block(x,32,64)
    
    x = convolutional_block(x,1,64,128)
    x = identity_block(x,64,128)
    x = identity_block(x,64,128)
    x = identity_block(x,64,128)
    
    x = convolutional_block(x,1,64,128)
    x = identity_block(x,64,128)
    x = identity_block(x,64,128)
    x = identity_block(x,64,128)
    x = identity_block(x,64,128)
    x = identity_block(x,64,128)
    
    x = convolutional_block(x,1,64,128)
    x = identity_block(x,64,128)
    x = identity_block(x,64,128)
    
    x = lay.SpatialPyramidPooling2D([32,16,1])(x)
    x = layers.Flatten()(x)
    x = layers.Dense(1)(x)
    x = layers.Activation('sigmoid')(x)
    
    model = tf.keras.Model(inputs=input_layer,outputs=x)
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    model.summary()
    model.fit(X_train,y_train,epochs=10,batch_size=256,verbose=1)
    loss,acc = model.evaluate(X_test,y_test)
    print("Loss : ",loss," Accuracy : ",acc)

Batch Processing Models

In [18]:
m = len(data[0])